In [1]:
import torch
from torch import Tensor
from jaxtyping import Float

a:Float[Tensor, "b t h"] = torch.randn(2, 3, 4)
b: Float[Tensor, "b t h"] = torch.randn(2, 3, 4)
attn: Float[Tensor, "b t"] = torch.randn(2, 3)
a_ref:  Float[Tensor, "b t h"] = torch.randn(2, 3, 4)
b_ref:  Float[Tensor, "b t h"] = torch.randn(2, 3, 4)
attn_ref: Float[Tensor, "b t"] = torch.randn(2, 3)

In [4]:
torch.log(torch.abs(a-b)/torch.abs(a_ref+b_ref)).mean()

tensor(-0.3211)

In [5]:
# should be closer
b2 = (a+b)/2
torch.log(torch.abs(a-b2)/torch.abs(a_ref+b_ref)).mean()

tensor(-1.0143)

: 